### Yelp APIs

In [1]:
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib

from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [2]:
Client_ID = "fhlzrhvkh8eY1FKkynuFQw"
API_KEY = "fAhmb4IfKarzC0TBkfSTbZVRpkdofHAVpMyZvRGbmAJ3yI9i-mCHEPCPzQtrmOaxtlXoYK7Qlzg-ZuzGUgZBwZ8f1soAqiqSEo6P2lcE4j5nkgHIzd6TIVyuSsDIXHYx" #  Replace this with your real API key

API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/' 
REVIEWS_PATH = '/reviews'

def request(host, path, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % API_KEY,
    }

    print('Querying {} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(term, location, limit=3):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': limit
    }
    return request(API_HOST, SEARCH_PATH, url_params=url_params)


def get_business(business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path)


def get_reviews(business_id):
    
    reviews_path = BUSINESS_PATH + business_id + REVIEWS_PATH
    
    return request(API_HOST, reviews_path)


def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)


### Data - Restaurant Week 2018  (https://www.kaggle.com/popoandrew/restaurant-week-2018)

In [3]:
import pandas as pd
import json

In [4]:
rstw_2018 = pd.read_csv('data/rstw_2018.csv')

# remove 57, 338 since they are not in yelp database
rstw_2018.drop(57, inplace=True)
rstw_2018.drop(338, inplace=True)
rstw_2018 = rstw_2018.reset_index(drop=True)

display(rstw_2018)

,name,street_address,google_map,review_count,phone,website,restaurant_type,average_review,food_review,service_review,...,star_1,star_2,star_3,star_4,star_5,description,restaurant_main_type,latitude,longitude,postal_code
0,Smoke Jazz and Supper Club,"2751 Broadway New York, NY 10025",//www.google.com/maps/search/?api=1&query=40.8...,2155,(212) 864-6662,http://www.smokejazz.com/,Contemporary American,4.421550,3.9,4.2,...,2,7,5,24,62,Smoke has augmented its reputation as one of M...,Contemporary American,40.801126,-73.968192,10025.0
1,Tavern on the Green,"1 Tavern on the Green New York, NY 10023",//www.google.com/maps/search/?api=1&query=40.7...,7029,(212) 877-8684,http://www.tavernonthegreen.com/,American,4.670290,4.4,4.4,...,1,2,6,19,72,Magical is a word thrown around a lot when dis...,American,40.772187,-73.977722,10023.0
2,ABC Kitchen,"35 East 18th Street New York, NY 10003",//www.google.com/maps/search/?api=1&query=40.7...,6031,(212) 475-5829,http://www.abckitchennyc.com/,Contemporary American,4.760310,4.6,4.4,...,1,2,5,15,77,ABC Kitchen with Jean-Georges: passionately co...,Contemporary American,40.737897,-73.989499,10003.0
3,Catch New York,"21 Ninth Avenue New York, NY 10014",//www.google.com/maps/search/?api=1&query=40.7...,4830,(212) 392-5978,http://www.catchhg.com/,Seafood,4.548300,4.6,4.2,...,3,2,6,15,74,CATCH features contemporary renditions of clas...,Seafood,40.740227,-74.006340,10014.0
4,Becco,"355 West 46th Street New York, NY 10010",//www.google.com/maps/search/?api=1&query=40.7...,18139,(212) 397-7597,http://www.becco-nyc.com/,Italian,4.418139,4.4,4.4,...,1,3,9,27,60,Lidia Bastianich & her son Joe's Theater Distr...,Italian,40.760804,-73.989832,10010.0
5,Estiatorio Milos - NY,"125 West 55th Street New York, NY 10019",//www.google.com/maps/search/?api=1&query=40.7...,4404,(212) 245-7400,http://www.estiatoriomilos.com/,Seafood,4.844040,4.8,4.6,...,1,1,3,14,81,"The freshest, most flavorful fish, vegetables ...",Seafood,40.763598,-73.978951,10019.0
6,Tao Uptown,"42 East 58th Street New York, NY 10022",//www.google.com/maps/search/?api=1&query=40.7...,10600,(212) 888-2288,http://www.taorestaurant.com/,Asian,4.610600,4.5,4.4,...,1,2,7,21,69,"A 16-foot Buddha, which floats above a reflect...",Asian,40.762607,-73.971478,10022.0
7,Red Rooster Harlem,"310 Lenox Avenue New York, NY 10027",//www.google.com/maps/search/?api=1&query=40.8...,5682,(212) 792-9001,http://redroosterharlem.com/,Southern,4.456820,4.4,4.2,...,2,3,10,24,61,"Located in the heart of Harlem, Red Rooster se...",Other,40.808057,-73.944914,10027.0
8,Del Frisco's Double Eagle Steak House - New Yo...,"1221 Avenue of the Americas New York, NY 10020",//www.google.com/maps/search/?api=1&query=40.7...,7225,(212) 575-5129,http://delfriscos.com/,Steakhouse,4.772250,4.6,4.7,...,1,2,5,16,76,A midtown classic with sprawling views of the ...,Steakhouse,40.759425,-73.982222,10020.0
9,FIG & OLIVE Fifth Avenue,"10 East 52nd Street New York, NY 10021",//www.google.com/maps/search/?api=1&query=40.7...,7186,(212) 319-2002,http://www.figandolive.com/,Mediterranean,4.571860,4.5,4.3,...,1,3,7,25,64,FIG & OLIVE is about passion for the best oliv...,Mediterranean,40.759280,-73.975634,10021.0


In [5]:
def processName(name):
    return name.split('- ')[0].replace("'s", "").replace(",", "").replace("& ", "").replace("Midtown", "").replace("Times Square", "").replace("Caberet", "Cabaret").strip()


In [6]:
# i = 0
# res = search(processName(rstw_2018.loc[i]['name']), rstw_2018.loc[i]['street_address'], limit=1)
# print(rstw_2018.loc[i]['name'], "=>", processName(rstw_2018.loc[i]['name']))
# print(res)

### Get restaurant profile through Yelp APIs

In [7]:
# nyc_rsts = []
# for i in range(rstw_2018.shape[0]):
#     result = search(processName(rstw_2018.loc[i]['name']), rstw_2018.loc[i]['street_address'], 1)['businesses']
#     if result:
#         nyc_rsts.append(result[0])
#     else:
#         print("Missing:", name)

In [9]:
# df_nyc_rsts = pd.read_json(json.dumps(nyc_rsts))
# display(df_nyc_rsts)
# df_nyc_rsts.to_csv("data/yelp_rst_2019.csv", index=False)

In [10]:
yelp_rsts = pd.read_csv("data/yelp_rst_2019.csv")
display(yelp_rsts)

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,smoke-jazz-and-supper-club-new-york-2,"[{'alias': 'jazzandblues', 'title': 'Jazz & Bl...","{'latitude': 40.8012384, 'longitude': -73.9681...",(212) 864-6662,5.912356,trbr9pYDDmXCj9z8m2d6LQ,https://s3-media2.fl.yelpcdn.com/bphoto/mtoknH...,False,"{'address1': '2751 Broadway', 'address2': None...",Smoke Jazz & Supper Club,1.212865e+10,$$,4.0,352,"['pickup', 'delivery']",https://www.yelp.com/biz/smoke-jazz-and-supper...
1,tavern-on-the-green-new-york-7,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 40.772194, 'longitude': -73.9776}",(212) 877-8684,20.401585,hKdvsbuQRtjajWYwjaoOQw,https://s3-media1.fl.yelpcdn.com/bphoto/fBgjO7...,False,"{'address1': '67 Central Park W', 'address2': ...",Tavern on the Green,1.212878e+10,$$$,3.5,1240,[],https://www.yelp.com/biz/tavern-on-the-green-n...
2,abc-kitchen-new-york,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 40.738085, 'longitude': -73.989472}",(212) 475-5829,39.424873,dMhRafXdr765DHe0k-QfaQ,https://s3-media2.fl.yelpcdn.com/bphoto/k2V0Qy...,False,"{'address1': '35 E 18th St', 'address2': '', '...",ABC Kitchen,1.212476e+10,$$$,4.0,2866,[],https://www.yelp.com/biz/abc-kitchen-new-york?...
3,catch-new-york-new-york,"[{'alias': 'advertising', 'title': 'Advertisin...","{'latitude': 40.74681, 'longitude': -73.98442}",(212) 715-8700,1994.761480,7mB_COyhOY1k01JcOu-wTw,https://s3-media4.fl.yelpcdn.com/bphoto/q1THJC...,False,"{'address1': '15 E 32nd St', 'address2': '4th ...",Catch -New York,1.212716e+10,NaN,3.5,37,[],https://www.yelp.com/biz/catch-new-york-new-yo...
4,becco-new-york-4,"[{'alias': 'italian', 'title': 'Italian'}]","{'latitude': 40.76083, 'longitude': -73.98981}",(212) 397-7597,3.072498,QObHX0yR6zd0WfksRDbJTA,https://s3-media3.fl.yelpcdn.com/bphoto/w5_bcc...,False,"{'address1': '355 W 46th St', 'address2': '', ...",Becco,1.212398e+10,$$$,3.5,2680,[],https://www.yelp.com/biz/becco-new-york-4?adju...
5,estiatorio-milos-new-york,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 40.76368, 'longitude': -73.97922}",(212) 245-7400,21.007385,mhhTORjUdXj-UIDjyGmhBg,https://s3-media1.fl.yelpcdn.com/bphoto/MHD33q...,False,"{'address1': '125 W 55th St', 'address2': '', ...",Estiatorio Milos,1.212246e+10,$$$$,4.0,836,[],https://www.yelp.com/biz/estiatorio-milos-new-...
6,tao-uptown-new-york,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...","{'latitude': 40.76269, 'longitude': -73.97147}",(212) 888-2288,1.660646,TU_BU9HLflYI2xlyqVQdRA,https://s3-media4.fl.yelpcdn.com/bphoto/Hlkr0I...,False,"{'address1': '42 E 58th St', 'address2': '', '...",TAO Uptown,1.212888e+10,$$$,3.5,2462,"['pickup', 'delivery']",https://www.yelp.com/biz/tao-uptown-new-york?a...
7,red-rooster-harlem-new-york-3,"[{'alias': 'tradamerican', 'title': 'American ...","{'latitude': 40.80815, 'longitude': -73.94488}",(212) 792-9001,6.188861,fxGpXRxFUDzlU3Cyszu4uQ,https://s3-media3.fl.yelpcdn.com/bphoto/S_4wwM...,False,"{'address1': '310 Lenox Ave', 'address2': '', ...",Red Rooster Harlem,1.212793e+10,$$$,3.5,2952,[],https://www.yelp.com/biz/red-rooster-harlem-ne...
8,del-friscos-double-eagle-steakhouse-new-york,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...","{'latitude': 40.759412, 'longitude': -73.982221}",(212) 575-5129,8.690020,b8a-8u_A51v2IzyjLVsx6w,https://s3-media3.fl.yelpcdn.com/bphoto/FyQ0V-...,False,"{'address1': '1221 Ave Of The Americas', 'addr...",Del Frisco's Double Eagle Steakhouse,1.212576e+10,$$$$,4.5,2763,[],https://www.yelp.com/biz/del-friscos-double-ea...
9,fig-and-olive-fifth-ave-new-york-4,"[{'alias': 'mediterranean', 'title': 'Mediterr...","{'latitude': 40.7594108995958, 'longitude': -7...",(212) 319-2002,21.674110,FON3TceJNsTC0I0mzBwOKg,https://s3-media1.fl.yelpcdn.com/bphoto/ZE51Xe...,False,"{'address1': '10 E 52nd St', 'address2': '', '...",Fig & Olive Fifth Ave,1.212319e+10,$$$,3.5,1103,"['delivery', 'pickup

#### Check correctness

In [11]:
for i in range(yelp_rsts.shape[0]):
    print(i, rstw_2018.loc[i]['name'], "=>", yelp_rsts.loc[i]['name'])

0 Smoke Jazz and Supper Club => Smoke Jazz & Supper Club
1 Tavern on the Green => Tavern on the Green
2 ABC Kitchen => ABC Kitchen
3 Catch New York => Catch -New York
4 Becco => Becco
5 Estiatorio Milos - NY => Estiatorio Milos
6 Tao Uptown => TAO Uptown
7 Red Rooster Harlem => Red Rooster Harlem
8 Del Frisco's Double Eagle Steak House - New York City => Del Frisco's Double Eagle Steakhouse
9 FIG & OLIVE Fifth Avenue => Fig & Olive Fifth Ave
10 Rosa Mexicano by Lincoln Center => Rosa Mexicano
11 ABC Cocina => ABC Cocina
12 Gallaghers Steakhouse - Manhattan => Gallaghers Steakhouse
13 Boulud Sud => Boulud Sud
14 Nougatine at Jean Georges => Nougatine at Jean-Georges
15 Sarabeth's Central Park South => Sarabeth's
16 Tao Downtown => TAO Downtown
17 ilili => ilili
18 Scarpetta => Scarpetta
19 Atlantic Grill Near Lincoln Center => Atlantic Grill
20 Empellon Midtown => Empellón
21 Ocean Prime - New York => Ocean Prime
22 Sarabeth's West => Sarabeth's West
23 STK - NYC - Midtown => STK Midtow

300 Noreetuh => Noreetuh
301 El Toro Blanco => El Toro Blanco
302 SAN CARLO  Osteria Piemonte => San Carlo Osteria Piemonte
303 The Library at the Public => The Library at The Public
304 CUT By Wolfgang Puck at FS Downtown New York => Wolfgang Puck
305 Osteria della Pace - Eataly NYC Downtown => Osteria della Pace
306 Tuome => Tuome
307 Barano => Barano
308 Buttermilk Channel => Buttermilk Channel
309 La Loteria => La Loteria
310 Il Mulino Prime => Il Mulino Prime
311 Socarrat Paella Bar - Nolita => Socarrat Paella Bar - Chelsea
312 La Sirene => La Sirene
313 Fonda - East Village => Fonda East Village
314 Kingsley => Kingsley
315 Porsena => Porsena
316 The Palm Tribeca => The Palm Tribeca
317 Nickel & Diner => Nickel & Diner
318 David Burke Kitchen => David Burke Kitchen
319 Il Cortile Restaurant => Il Cortile Restaurant
320 Felice 15 Gold Street => Felice
321 Cipriani - Wall Street => Cipriani
322 Rosa Mexicano - TriBeCa => Rosa Mexicano
323 North Square => North Square
324 Edi and Th

### Get reviews from Yelp provided data set (FAIL -> no NYC data available)

In [12]:
# i = 0
# res = get_reviews(yelp_rsts.loc[i]['id'])
# print(yelp_rsts.loc[i]['name'], "=>", yelp_rsts.loc[i]['review_count'])
# print(res)

In [13]:
# %%time

# reviews = pd.read_csv("data/review.csv")

In [14]:
# reviews.head()

In [15]:
# %%time

# reviews.sort_values(by=['business_id', 'date'], inplace=True)

In [16]:
# display(reviews)

In [17]:
# count = 0
# for i in range(yelp_rsts.shape[0])[:]:
#     b_id = yelp_rsts.loc[i]['id']
#     start = reviews['business_id'].searchsorted(b_id)
#     end = reviews['business_id'].searchsorted(b_id, side='right')
#     if start == end:
#         print("Not exists:", b_id, reviews.loc[start]['business_id'])
#         count += 1
#     print(start, end)
# print(count, yelp_rsts.shape[0])

In [18]:
# res = get_business('--1UhMGODdWsrMastO9DZw')
# print(res)

### Yelp Review Scraper

In [19]:
from bs4 import BeautifulSoup
import requests
import json
import time

In [20]:
def scrape_content(content):
    user_content = content.find('div', class_='media-story')
    username = user_content.find('a', class_='user-display-name')
    username = username.get_text().strip() if username else "_ghost"
    user_loc = user_content.find('li', class_='user-location')
    user_loc = user_loc.find('b').get_text().strip() if username else ""
    user_friends = user_content.find('li', class_='friend-count')
    user_friends = user_friends.find('b').get_text().strip() if user_friends else 0
    user_reviews = user_content.find('li', class_='review-count')
    user_reviews = user_reviews.find('b').get_text().strip() if user_reviews else 0
    user_photos = user_content.find('li', class_='photo-count')
    user_photos = user_photos.find('b').get_text().strip() if user_photos else 0
    
    review_content = content.find('div', class_='review-wrapper')
    rating = review_content.find('div', class_='i-stars')['title'].split(" ")[0].strip()
    time_created = review_content.find('span', class_='rating-qualifier').get_text().replace('\n', '').strip().split(' ')[0]
    text = review_content.find('p').get_text().strip()
    useful = review_content.find('a', class_='useful')
    if useful:
        useful = useful.find('span', class_='count').get_text().strip()
    useful = useful if useful else 0
    funny = review_content.find('a', class_='funny')
    if funny:
        funny = funny.find('span', class_='count').get_text().strip()
    funny = funny if funny else 0
    cool = review_content.find('a', class_='cool')
    if cool:
        cool = cool.find('span', class_='count').get_text().strip()
    cool = cool if cool else 0
    
    return [rating, time_created, text, useful, funny, cool, username, user_loc, user_friends, user_reviews, user_photos]
    

In [21]:
def get_proxies():
    url = 'https://www.us-proxy.org/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    rows = soup.find('tbody').find_all('tr')
    proxies = set()
    for row in rows:
        cols = row.find_all('td')
        if cols[4].get_text() != 'elite proxy':
            continue
        proxies.add(cols[0].get_text() + ":" + cols[1].get_text())

    return proxies

In [22]:
def run_scrapper(start, end):
    yelp_rsts = pd.read_csv("data/yelp_rst_2019.csv")

    PER_PAGE = 20

    proxies = set()

    for i in range(yelp_rsts.shape[0])[max(0, start): min(yelp_rsts.shape[0], end)]:
        hasNextPage = True
        start = 0
        url = yelp_rsts.loc[i]['url'].split("?")[0] + "?sort_by=date_desc&start="
        print("=========================", i, yelp_rsts.loc[i]['alias'], "=======================")
        results = pd.DataFrame(data=[], columns=['rating', 'time_created', 'text', 'useful', 'funny', 'cool', 'username', 'user_loc', 'user_friends', 'user_reviews', 'user_photos'])

        while hasNextPage:
            page_url = url + str(start)
            print(page_url)
            success = False
            fails = set()
            if not proxies:
                proxies = get_proxies()
                print("New proxies:", len(proxies))
            
            success_proxy = ""
            for proxy in proxies:
                try:
#                     req = requests.get(page_url, timeout=30)
                    req = requests.get(page_url, proxies={"http": proxy, "https": proxy}, timeout=30)
                    if req.status_code >= 300:
                        raise Exception(req.status_code)
                    else:
                        success_proxy = proxy
                        break
                except:
                    fails.add(proxy)
                    print("Fail proxy:", proxy, ", remains:", len(proxies) - len(fails))
                    

            print("Fail proxies:", len(fails))
            proxies -= fails
            if not success_proxy:    
                continue

            soup = BeautifulSoup(req.content, 'html.parser')
            pageofpages = soup.find('div', class_='page-of-pages')
            if not pageofpages:
                proxies.remove(success_proxy)
                continue

            pageofpages = pageofpages.get_text().strip().replace('Page ', '').split(' of ')
            hasNextPage = pageofpages[0] != pageofpages[1]

            for content in soup.find_all('div', class_='review review--with-sidebar'):
                results = results.append(pd.Series(scrape_content(content), index=results.columns), ignore_index=True)   
            
            start += PER_PAGE

            print("results len:", results.shape[0], "; hasNext:", hasNextPage)
            time.sleep(0.5)

        filename = "data/reviews/{:03d}_{}.csv".format(i, yelp_rsts.loc[i]['alias'].replace("-", "_"))
        results.to_csv(filename, index=False)
        print("Export to", filename)

In [23]:
# run_scrapper(213, 214)

#### Check correctness

In [36]:
from os import listdir
from os.path import isfile, join

dir_path = "data/reviews"
review_files = [f for f in listdir(dir_path) if f.endswith('.csv') and isfile(join(dir_path, f))]

minors = 0
majors = 0
for f in review_files:
    idx = int(f.split('_')[0])
    reviews = pd.read_csv(join(dir_path, f))
    diff = yelp_rsts.loc[idx]['review_count'] - reviews.shape[0]
    if diff != 0:
        if abs(diff) > 3 and abs(diff) / reviews.shape[0] > 0.05:
            majors += 1
            print("MAJOR DIFF")
        else:
            minors += 1
        print(idx, yelp_rsts.loc[idx]['alias'], yelp_rsts.loc[idx]['review_count'], reviews.shape[0], '=>', yelp_rsts.loc[idx]['review_count'] - reviews.shape[0])

print("===============================================")
print("Major diffs: {}; Minor diffs: {}; Total: {}".format(majors, minors, len(review_files)))



0 smoke-jazz-and-supper-club-new-york-2 352 350 => 2
1 tavern-on-the-green-new-york-7 1240 1238 => 2
2 abc-kitchen-new-york 2866 2855 => 11
3 catch-new-york-new-york 37 36 => 1
4 becco-new-york-4 2680 2674 => 6
5 estiatorio-milos-new-york 836 827 => 9
6 tao-uptown-new-york 2462 2449 => 13
7 red-rooster-harlem-new-york-3 2952 2939 => 13
8 del-friscos-double-eagle-steakhouse-new-york 2763 2751 => 12
9 fig-and-olive-fifth-ave-new-york-4 1103 1098 => 5
10 rosa-mexicano-new-york-6 1117 1111 => 6
11 abc-cocina-new-york 842 839 => 3
12 gallaghers-steakhouse-new-york 986 970 => 16
13 boulud-sud-new-york 684 682 => 2
14 nougatine-at-jean-georges-new-york-3 1072 1061 => 11
20 empellón-new-york-3 297 299 => -2
25 blue-fin-new-york 946 944 => 2
26 bar-boulud-new-york-2 819 816 => 3
27 hakkasan-new-york 850 846 => 4
28 lincoln-ristorante-new-york 543 541 => 2
29 cafe-boulud-new-york-4 672 670 => 2
30 the-view-restaurant-and-lounge-new-york 995 974 => 21
31 park-avenue-autumn-winter-spring-summer-ne

287 root-and-bone-new-york 1478 1475 => 3
290 american-cut-steakhouse-tribeca-new-york 537 536 => 1
291 khe-yo-new-york 654 653 => 1
292 toloache-thompson-new-york-3 418 417 => 1
293 nobu-downtown-new-york 294 292 => 2
295 temple-court-new-york 181 179 => 2
296 pera-soho-new-york 427 424 => 3
297 tribeca-grill-new-york-2 579 577 => 2
299 dos-caminos-new-york-2 1479 1473 => 6
302 san-carlo-osteria-piemonte-new-york-city 175 172 => 3
306 tuome-new-york-2 413 412 => 1
308 buttermilk-channel-brooklyn 1815 1811 => 4
309 la-loteria-new-york 291 290 => 1
311 socarrat-paella-bar-chelsea-new-york-3 1097 1095 => 2
312 la-sirene-new-york 1561 1557 => 4
316 the-palm-tribeca-new-york-2 262 260 => 2
317 nickel-and-diner-new-york-2 371 370 => 1
318 david-burke-kitchen-new-york 1122 1121 => 1
319 il-cortile-restaurant-new-york 457 456 => 1
320 felice-new-york-6 312 310 => 2
322 rosa-mexicano-new-york-3 1411 1410 => 1
324 edi-and-the-wolf-new-york 408 407 => 1
325 gran-morsi-new-york 308 306 => 2
326 b

In [31]:
print(yelp_rsts.loc[77]['url'])

https://www.yelp.com/biz/tbar-new-york-2?adjust_creative=fhlzrhvkh8eY1FKkynuFQw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=fhlzrhvkh8eY1FKkynuFQw
